In [2]:
from Crypto.Cipher import AES

def AES_128(key: bytearray, message: bytearray) -> bytearray:
    a = AES.new(key, AES.MODE_ECB)
    return bytearray(a.encrypt(message))

In [205]:
def pad(message,l_block):
    message_len = len(message)
    lista_numeros = []
    agregar = 0
    for i in range(0,message_len,l_block):
        lista_numeros.append([message[i:i + l_block]])
    if len(lista_numeros)!=0:
        largo_ultimo = len(lista_numeros[-1][0])
        if largo_ultimo < l_block:
            agregar = l_block - largo_ultimo 
        print(agregar)
        if agregar != 0:
            message += int(1).to_bytes(agregar,'little')
    
    result = int(message_len).to_bytes(l_block,'big')
    message += result
    return message

In [207]:
def davies_meyer(encrypt,l_key,l_message):
    def compression(message):
        key = message[0:l_key]
        ms = message[l_key:l_key+l_message]
        hash_0 = encrypt(key,ms)
        hash_1 = int.from_bytes(hash_0, byteorder="big") ^ int.from_bytes(ms, byteorder="big")

        return hash_1.to_bytes(max(len(hash_0), len(ms)), byteorder="big")
    return compression

In [208]:
compresion = davies_meyer(AES_128, 16, 16)
compresion(bytearray(b'123456789012345601234567890123456'))

b'Kz\xce\xba\xe9"6"|\xbb\x8cO\x84\xb5\x8f;'

In [209]:
def  merkle_damgard(iv, compresion, l_block):
    def hash(message):
        lista_numeros = []
        mesg_pad = pad(message,l_block)
        message_len = len(mesg_pad)
        
        for i in range(0,message_len,l_block):
            lista_numeros.append(bytearray(mesg_pad[i:i + l_block]))

        byte_inicial = lista_numeros[0] + iv
        hash_inicial = compresion(byte_inicial)
        for i in range(1,len(lista_numeros)):
            byte_aux = lista_numeros[i] + hash_inicial
            hash_inicial = compresion(byte_aux)
            
        return hash_inicial
    
    return hash

In [212]:
if __name__ == "__main__":
    compresion = davies_meyer(AES_128, 16, 16)
    hash = merkle_damgard(bytearray(b'1234567890123456'), compresion, 16)

    s1 = bytearray(b'Este es un mensaje de prueba para la tarea 2')
    s2 = bytearray(b'Este es un mensaje de Prueba para la tarea 2')
    s4 = bytearray(b'')
    s3 = bytearray(b'Un mensaje corto')


    h1 = hash(s1)
    h2 = hash(s2)
    h4 = hash(s4)
    h3 = hash(s3)
   

    print(h1)
    print(h2)
    print(h4)
    print(h3)
    

4
4
0
b'\xe9\xe8\xac\x12\\\xf2\xc8\x16\xceOV\xc5Y.T\xea'
b'\xb6\xfb\xc6a\x12\xae\x95\x1f\xda\xc5\x13\xde\x06|Q\x96'
b'p\xca \xd8\x9c\xeb\xe6\xb1\xce\xcf\x03\xb2\x9e\x93\x19\xbc'
b'\xc5\xec\xcdd\xa4(R*\xf0L*QtL\xda\x81'


In [3]:
class RSASender:
    def ___init__(self, public_key):
        self.public_key = public_key

    def encrypt(self,message):
        len_e = self.public_key[0:4]
        len_e = int.from_bytes(len_e, 'big')